Dr. Probe Python Wrapper.ipynb contains multislice simulation code for LACBED and unfinished multislice template matching code

In [1]:
import os

In [2]:
#Change operating directory to where the CLT is
os.chdir('./DrProbeCLT-Winx64')

In [1]:
# Import necesary packages 
import numpy as np
import matplotlib.pyplot as plt
import drprobe as drp
import math
from itertools import product 
from time import time
from scipy import ndimage
%matplotlib qt

In [57]:
# Set up phase gratings
nx=540
ny=243
nz = 6

In [4]:
#Projection of new cell in desired zone axis
prj=(0,1,0,1,0,3,6.35674,6.35674,0.895) #First 3 numbers=Projection in x-axis. Following 3=Projection in y-axis. Last 3=new supercell size
tla=(0,0,0) #Tilting multiplication along x,y,z for discretisation purpose

In [5]:
#Celslc to orient cif file in desired zone axis. Notice=This function never works because of the space sent to the CLT from the prj variable. It is automatically assigned by the wrapper.
drp.commands.celslc(cel_file='./pigeonite.cif', # location of cel/cif file
                    slice_name='pigeonite', # target file names
                    ht=300,       # high tension
                    nx=512,        # number of sampling points along x
                    ny=512,        # number of sampling points along y
                    nz=6,        # number of sampling points along z (number of slices)
                    absorb=False,  # apply absorptive form factors
                    dwf=False,     # apply Debye-Waller factors
                    fl=False,
                    output=True,  # Command line output (prints executed command)
                    pot=False,      # Saves potentials
                    prj=prj,
                    tla=tla,
                    rti=True
                   )

Performed celslc with the following command:
 celslc -cif C:/Users/cqred/OneDrive/Documents/NTU/Y3S2/Project/drprobe_python_wrapper/pigeonite.cif  -slc pigeonite -nx 512 -ny 512 -nz 6 -ht 300 -rti -prj 0, 1, 0, 1, 0, 3, 6.35674, 6.35674, 0.895 -tla 0, 0, 0

 +---------------------------------------------------+
 | Program [celslc]                                  |
 | Version: 1.0.3 64-bit  -  2020 August 12          |
 | Author : Dr. J. Barthel, ju.barthel@fz-juelich.de |
 |          Forschungszentrum Juelich GmbH, GERMANY  |
 | License: GNU GPL 3 <http://www.gnu.org/licenses/> |
 +---------------------------------------------------+

 CELSLC parameters
  -cel <string> = CEL structure file (e.g. "atoms.cel")
  -cif <string> = CIF structure file (e.g. "atoms.cif")
  -ht <number>  = electron energy in keV (10 ... 1300)
  -nx <number>  = horizontal cell sampling (32 ... 8192)
  -ny <number>  = vertical cell sampling (32 ... 8192)
  -slc <string> = slice file name (e.g. "scl")
 [-abf <num

In [59]:
# Display potentials
slc_1 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_001.pot', dtype='complex64').reshape((nx, ny))
slc_2 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_002.pot', dtype='complex64').reshape((nx, ny))
slc_3 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_003.pot', dtype='complex64').reshape((nx, ny))
slc_4 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_004.pot', dtype='complex64').reshape((nx, ny))
slc_5 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_005.pot', dtype='complex64').reshape((nx, ny))
slc_6 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_006.pot', dtype='complex64').reshape((nx, ny))

In [70]:
fig, axs = plt.subplots(1, 6, figsize=(25, 15))
axs[0].imshow(np.real(slc_1))
axs[1].imshow(np.real(slc_2))
axs[2].imshow(np.real(slc_3))
axs[3].imshow(np.real(slc_4))
axs[4].imshow(np.real(slc_5))
axs[5].imshow(np.real(slc_6))
axs[0].set_title('Slice 1')
axs[1].set_title('Slice 2')
axs[2].set_title('Slice 3')
axs[3].set_title('Slice 4')
axs[4].set_title('Slice 5')
axs[5].set_title('Slice 6')

Text(0.5, 1.0, 'Slice 6')

In [48]:
#Set thickness list
thickness_list=np.arange(50,151,5)
for i,th in enumerate(thickness_list):
    slc_list=thickness_list
    slc_list[i]=math.floor(thickness_list[i]/(0.895/6))
thickness_list=np.arange(50,151,5)
thickness_list

array([ 50,  55,  60,  65,  70,  75,  80,  85,  90,  95, 100, 105, 110,
       115, 120, 125, 130, 135, 140, 145, 150])

In [29]:
#List of tilting angle
tilt_list=np.linspace(-1,1,11,dtype='float32')
tilt_list=list(product(tilt_list,repeat=2))
for tilt in tilt_list:
    print(tilt[0],tilt[1])

-1.0 -1.0
-1.0 -0.8
-1.0 -0.6
-1.0 -0.4
-1.0 -0.2
-1.0 0.0
-1.0 0.2
-1.0 0.4
-1.0 0.6
-1.0 0.8
-1.0 1.0
-0.8 -1.0
-0.8 -0.8
-0.8 -0.6
-0.8 -0.4
-0.8 -0.2
-0.8 0.0
-0.8 0.2
-0.8 0.4
-0.8 0.6
-0.8 0.8
-0.8 1.0
-0.6 -1.0
-0.6 -0.8
-0.6 -0.6
-0.6 -0.4
-0.6 -0.2
-0.6 0.0
-0.6 0.2
-0.6 0.4
-0.6 0.6
-0.6 0.8
-0.6 1.0
-0.4 -1.0
-0.4 -0.8
-0.4 -0.6
-0.4 -0.4
-0.4 -0.2
-0.4 0.0
-0.4 0.2
-0.4 0.4
-0.4 0.6
-0.4 0.8
-0.4 1.0
-0.2 -1.0
-0.2 -0.8
-0.2 -0.6
-0.2 -0.4
-0.2 -0.2
-0.2 0.0
-0.2 0.2
-0.2 0.4
-0.2 0.6
-0.2 0.8
-0.2 1.0
0.0 -1.0
0.0 -0.8
0.0 -0.6
0.0 -0.4
0.0 -0.2
0.0 0.0
0.0 0.2
0.0 0.4
0.0 0.6
0.0 0.8
0.0 1.0
0.2 -1.0
0.2 -0.8
0.2 -0.6
0.2 -0.4
0.2 -0.2
0.2 0.0
0.2 0.2
0.2 0.4
0.2 0.6
0.2 0.8
0.2 1.0
0.4 -1.0
0.4 -0.8
0.4 -0.6
0.4 -0.4
0.4 -0.2
0.4 0.0
0.4 0.2
0.4 0.4
0.4 0.6
0.4 0.8
0.4 1.0
0.6 -1.0
0.6 -0.8
0.6 -0.6
0.6 -0.4
0.6 -0.2
0.6 0.0
0.6 0.2
0.6 0.4
0.6 0.6
0.6 0.8
0.6 1.0
0.8 -1.0
0.8 -0.8
0.8 -0.6
0.8 -0.4
0.8 -0.2
0.8 0.0
0.8 0.2
0.8 0.4
0.8 0.6
0.8 0.8
0.8 1.0
1.0 -1.0
1.0 -0

In [56]:
#Directory of slices, to be used later
pigeonite010_dir='C:/Users/cqred/Documents/pigeonite010/slc2/pigeonite010'
augite010_dir='C:/Users/cqred/Documents/augite010/slc2/augite010'
slcdirectory=[pigeonite010_dir,augite010_dir]
slcdirectory

['C:/Users/cqred/Documents/pigeonite010/slc2/pigeonite010',
 'C:/Users/cqred/Documents/augite010/slc2/augite010']

In [58]:
#MSA directory,but without the msa.prm
pigeonite010_savedir='C:/Users/cqred/Documents/pigeonite010/msa/'
augite010_savedir='C:/Users/cqred/Documents/augite010/msa/'
savedirectory=[pigeonite010_savedir,augite010_savedir]
savedirectory

['C:/Users/cqred/Documents/pigeonite010/msa/',
 'C:/Users/cqred/Documents/augite010/msa/']

In [31]:
# Create Parameter file for multislice
msa = drp.msaprm.MsaPrm()

In [60]:
# Setup (fundamental) MSA parameters
for savedir,slcdir in zip(savedirectory,slcdirectory):
    for slices,th in zip(slc_list,thickness_list):
        for tilt in tilt_list:
            msa.conv_semi_angle=3.0 #Convergence beam semi-angle
            msa.inner_radius_ann_det=0
            msa.outer_radius_ann_det=100
            msa.detector=0,'0'
            msa.wavelength = 0.001969  # wavelength (in nm) for 300 keV electrons
            msa.source_radius=0.4 #HWHM source profile radius
            msa.focus_spread=3
            msa.focus_spread_kernel_hw=2
            msa.focus_spread_kernel_size=7
            msa.number_of_aber=0
            msa.tilt_x=tilt[0] #Tilting angle in x-axis
            msa.tilt_y=tilt[1] #Tilting angle in y-axis
            msa.h_scan_offset=0
            msa.v_scan_offset=0
            msa.h_scan_frame_size=1
            msa.v_scan_frame_size=1
            msa.scan_frame_rot=0
            msa.scan_columns=1
            msa.scan_rows=1
            msa.spat_coherence_flag=1
            msa.super_cell_x=1
            msa.super_cell_y=1
            msa.slice_files =slcdir # location of phase gratings
            msa.number_of_slices =6 # Number of slices in phase gratings
            msa.number_frozen_lattice=50
            msa.min_num_frozen=1
            msa.det_readout_period =0 # Readout at max thickness only
            msa.tot_number_of_slices=slices # indicate thickness
            msa.save_msa_prm(savedir+'msa_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])+'.prm',output=True)

Parameters successfully saved to file 'C:/Users/cqred/Documents/pigeonite010/msa/msa_10nm_x0.0,y0.0.prm'!
Parameters successfully saved to file 'C:/Users/cqred/Documents/augite010/msa/msa_10nm_x0.0,y0.0.prm'!


In [1]:
phase=['pigeonite010','augite010']
phase

['pigeonite010', 'augite010']

In [63]:
#Execute library of multislice
tic=time()
for phase,savedir in zip(phase,savedirectory):
    for th in thickness_list:
        for tilt in tilt_list:
            drp.commands.msa(prm_file=savedir+'msa_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])+'.prm', 
                             output_file='C:/Users/cqred/Documents/'+phase+'/output/'+phase+'_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])+'.dat',
                             padif=True,
                             output=False)
'Simulation time: %.2fs'%(time()-tic)

'Simulation time: 3.18s'

In [9]:
#function to crop np array
def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [12]:
#Load .dat binary file of simulated STEM LACBED
img = np.fromfile('./pigeonite010_50nm_padif_tot_sl335.dat', dtype='float32')

In [60]:
ny,nx= 498,540

In [82]:
img=img.reshape((ny,nx))
img=np.append(img,img)
img=img.reshape((ny*2,nx))
img=np.append(img,img,axis=1)
img=np.flipud(img)

In [83]:
img=crop_center(img,nx,ny)

In [84]:
plt.imshow(img, cmap='gray',vmax=0.0005)

## Below is the code for Multislice template matching. However, it is not finished yet

In [ ]:
from pyxem.signals.indexation_results import TemplateMatchingResults

In [149]:
def correlate(signal,phase,savedirectory,slc_list,thickness_list,tilt_list,
        n_largest=3,
        method="fast_correlation",
        *args,
        **kwargs,):  
                    matches = signal.map(
                                correlate_library,phase,savedirectory,slc_list,thickness_list,tilt_list,                                
                                n_largest=n_largest,
                                method=method,
                                inplace=False,
                                **kwargs,)

                    matching_results = TemplateMatchingResults(matches)
                    # matching_results = transfer_navigation_axes(matching_results, signal)

        return matching_results

In [ ]:
def correlate_library(signal,phase,savedirectory,slc_list,thickness_list,tilt_list, n_largest, method,rotation_angle):

    top_matches = np.empty((len(library), n_largest, 4), dtype="object")

    #if method == "zero_mean_normalized_correlation":
        #nb_pixels = image.shape[0] * image.shape[1]
        #average_image_intensity = np.average(image)
        #image_std = np.linalg.norm(image - average_image_intensity)
    
    rot_saved=np.empty((n_largest, 1),dtype=int)
    corr_saved=np.zeros((n_largest, 1))
    title_saved=np.empty((n_largest, 1),dtype='U16')

    for phase,savedir in zip(phase,savedirectory):
        for th,slc in zip(thickness_list,slc_list):
            for tilt in tilt_list:
                template=np.fromfile('C:/Users/cqred/Documents/'+phase+'/output/'+phase+'_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])+'_padif_tot_sl'+slc+'.dat',
                             dtype='float32')
                title=phase+'_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])
                for angle in range(rotation_angle): #iterate through the rotations                                      
                        if method == "fast_correlation":
                            rotated_template = ndimage.rotate(template, angle, reshape=False, order=1)
                            pattern_norm=np.linalg.norm(rotated_template)
                            corr_local = fast_correlation(signal, rotated_template, pattern_norm)
                        if corr_local > np.min(corr_saved):
                            title_saved[np.argmin(corr_saved)]= title
                            rot_saved[np.argmin(corr_saved)]= angle
                            corr_saved[np.argmin(corr_saved)] = corr_local

                        combined_array = np.hstack((rot_saved,corr_saved,title_saved))
                        combined_array = combined_array[np.flip(combined_array[:, 3].argsort())] #sort array from smallest correlation value
                        top_matches[a, :, 0] = list(library)[a] #phase
                        top_matches[a, :, 1] = combined_array[:, 0] #thickness
                        top_matches[a, :, 2] = combined_array[:, 1] #tilting angle and axis
                        top_matches[a, :, 3] = combined_array[:, 2] #rotation angle
                        top_matches[a, :, 4] = combined_array[:, 3] #correlation

    return top_matches.reshape(-1, 4)

In [ ]:
dp = pxm.load('C:/Users/cqred/OneDrive/Documents/NTU/Y3S2/Project/DiffData/Diffraction SI.dm4')
dp.set_signal_type('electron_diffraction')
dp

In [ ]:
correlate(dp,phase,savedirectory,slc_list,thickness_list,tilt_list)

In [ ]:
#Template matching of variable 'data'
indexer = IndexationGenerator(data, diff_lib)
indexation_results = indexer.correlate(n_largest=3)